# Scrapen van schilderijen

Om een extra schilder toe te voegen aan ons project, moeten we eerst aan een hele hoeveelheid images geraken. Dit bekomen we door middel van scraping. Ik zal in deze notebook zoveel mogelijk images scrapen van de schilder Rembrandt. Ik zal de scraping uitvoeren op [deze](http://www.rembrandtpainting.net/complete_catalogue/complete_catalogue.htm) site.

In [3]:
import os, shutil
import requests
from bs4 import BeautifulSoup
import urllib.request

Nieuwe folder maken om de gescrapte images in op te slaan, we maken deze dan ook de actieve folder zodat de images dan ook daadwerkelijk hierin opgeslagen worden.

In [2]:
os.mkdir("../datasets/paintings/Rembrandt")      # aanmaken directory
os.chdir("../datasets/paintings/Rembrandt/")     # deze directory active maken, zodat de gescrapete images hierin terecht komen

## Eerste poging

Hieronder vindt u de scraping methode die ik ingediend heb voor de scraping challenge. Ik was echter niet tevreden met het aantal images die ik hiermee scrapte, dus heb ik het opnieuw geprobeerd in een poging toch meer images te kunnen scrapen. Deze tweede poging vindt u hieronder, dat is dan ook de methode die ik gebruikt heb voor de dataset van de Rembrandt schrilderijen die u vindt onder 'datasets/paintings/Rembrandt'.

In [4]:
time_period = ["1626-35.htm", "1636_54.htm", "1655-69.htm"]
full_links = []

for period in time_period: # iedere pagina van de verschillende tijdperioden bezoeken

  url_catalogue = "http://www.rembrandtpainting.net/rembrandt_painting_" + period # url samenvoegen

  response = requests.get(url_catalogue) # request doen naar gemaakte url
  soup = BeautifulSoup(response.content, "html.parser") # html content van pagina ophalen


  if response.status_code == 200: # indien een succesvolle statuscode (200)

    anchors = soup.find_all("a") # alle links ophalen
    for anchor in anchors:  # voor iedere link
      href = anchor["href"] # href van iedere <a> in de html
      helpert = ["1620", "1636", "1655"]
      for helper in helpert:
        if helper in href:
          full_links.append(f"http://www.rembrandtpainting.net/{href}") # pagina link samenstellen


In [4]:
images = []
helpers = ["rmbrndt_1620-35/", "rmbrndt_1636-1654/", "rmbrndt_1655-1669/"]
i = 0

for link in full_links: # voor iedere link in vorige stap

  response = requests.get(link)
  # print(link)
  soup = BeautifulSoup(response.content, "html.parser") # html ophalen

  for div in soup.find_all("div", attrs={"id": "workimage"}): # alle divs overlopen met id="workimage"

    try: # iedere mogelijke optie overlopen van link, indien error print fail

      if div.find("img")["src"].startswith("1620"):
        urllib.request.urlretrieve(f"http://www.rembrandtpainting.net/{helpers[0]}" + div.find("img")["src"], f"Rembrandt {i}.jpg")
      elif div.find("img")["src"].startswith("1636"):
        urllib.request.urlretrieve(f"http://www.rembrandtpainting.net/{helpers[1]}" + div.find("img")["src"], f"Rembrandt {i}.jpg")
      elif div.find("img")["src"].startswith("1655"):
        urllib.request.urlretrieve(f"http://www.rembrandtpainting.net/{helpers[2]}" + div.find("img")["src"], f"Rembrandt {i}.jpg")

      i += 1
    except:
      print("fail")

Met deze poging bekwam ik 120 gescrapte images.

In [5]:
print(i)

120


## Tweede poging

Hieronder vindt u mijn definitieve poging waarmee ik meer images kon scrapen.

In [5]:
"""
    Links ophalen van iedere categorie
"""
url_catalogue = "http://www.rembrandtpainting.net/complete_catalogue/complete_catalogue.htm"

response = requests.get(url_catalogue)
soup = BeautifulSoup(response.content, "html.parser")

full_links = []
if response.status_code == 200:

  anchors = soup.find_all("a")[11:18] # alleen de nodige <a> tags ophalen
  for anchor in anchors:
    href = anchor["href"]
    full_links.append(f"http://www.rembrandtpainting.net/complete_catalogue/{href}")

In [6]:
full_links

['http://www.rembrandtpainting.net/complete_catalogue/start_history.htm',
 'http://www.rembrandtpainting.net/complete_catalogue/start_history_two.htm',
 'http://www.rembrandtpainting.net/complete_catalogue/start_landscapes.htm',
 'http://www.rembrandtpainting.net/complete_catalogue/start_self_portraits.htm',
 'http://www.rembrandtpainting.net/complete_catalogue/start_portraits.htm',
 'http://www.rembrandtpainting.net/complete_catalogue/start_family_portraits.htm',
 'http://www.rembrandtpainting.net/complete_catalogue/start_allegories.htm']

In [7]:
url = "http://www.rembrandtpainting.net/complete_catalogue/"
basis_url = "http://www.rembrandtpainting.net/"

count = 0

for link in full_links: # voor iedere link in van de verschillende categorieën
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    page = soup.find_all('tr')

    for row in page:
        for anchor in row.find_all('a'):
            link_image_page = url + anchor['href']
            response = requests.get(link_image_page)
            soup = BeautifulSoup(response.content, 'html.parser')

            try:
                div = soup.find('div',attrs={'id' : 'workimage'}) # de div ophalen waarin de image zich bevindt
                img_src = div.find('img')['src'] # src tag van de image ophalen

                if "../../" in img_src:
                    img_link = basis_url + img_src[6:len(img_src)] # 6 is de index van de laatste slash in ../../

                else:
                    img_link = link_image_page[0:link_image_page.rfind("/")+1] + img_src



                file_name = f"Rembrandt {count}.jpg" # formaat van image naam --> Rembrandt 0.jpg, Rembrandt 1.jpg, ...
                urllib.request.urlretrieve(img_link, file_name) # downloaden van de image
                print(f"\t{img_link}")  # printen succesvol gedownloade images (met een tab ervoor om duidelijk onderscheid te maken tussen
                                        # succesvolle en gefaalde images
                count += 1

            except:
                print(f"FAIL") # printen gefaalde images


	http://www.rembrandtpainting.net/complete_catalogue/storia/images/st_stephen.jpg
	http://www.rembrandtpainting.net/rmbrndt_1620-35/1620_35_images/history_painting.jpg
	http://www.rembrandtpainting.net/complete_catalogue/storia/images/eunuch.jpg
	http://www.rembrandtpainting.net/rmbrndt_1620-35/1620_35_images/blind_hermit.jpg
	http://www.rembrandtpainting.net/complete_catalogue/storia/images/beating_ass.jpg
	http://www.rembrandtpainting.net/complete_catalogue/storia_b/images/rich_man.jpg
	http://www.rembrandtpainting.net/complete_catalogue/storia_b/images/paul_prison.jpg
	http://www.rembrandtpainting.net/complete_catalogue/../rmbrndt_1620-35/1620_35_images/earlywork.jpg
	http://www.rembrandtpainting.net/rmbrndt_1620-35/1620_35_images/earlywork.jpg
	http://www.rembrandtpainting.net/rmbrndt_1620-35/1620_35_images/dahlia.jpg
FAIL
	http://www.rembrandtpainting.net/complete_catalogue/storia/images/simon_baby_jesus.jpg
	http://www.rembrandtpainting.net/rmbrndt_1620-35/1620_35_images/judas_re

Aantal gescrapte images printen, we bekomen nu 254 images. Dit is al veel beter dan de 120 die ik in mijn eerste poging kreeg.

In [8]:
print(count)

254


Terug de project root folder de actieve folder maken

In [9]:
os.chdir("../../..")  # terug root directory van het project

In [10]:
os.getcwd()

'C:\\My Projects\\Opdracht-schilders'